In [1]:
%load_ext lab_black

In [2]:
import json
import math
import os
import random
import shutil
import sys
from collections import defaultdict
from dataclasses import dataclass
from datetime import datetime
from pathlib import Path
from typing import Callable, List, Sequence, Tuple

import albumentations as A
import cv2
import imageio
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from albumentations.pytorch import ToTensorV2
from PIL import Image
from sklearn.model_selection import KFold
from torch import nn, optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.utils.data import DataLoader, Dataset, Subset
from tqdm import tqdm

import networks
import utils
from error_list import error_list

---

## 하이퍼 파라미터

In [3]:
CHECKPOINT_PATH = Path("results/hrnet+det/ckpt-20210321-124655_2.pth")

POSE_MODEL = "HRNet-W48"
DET_PRETRAINED = ""
RESULT_DIR = Path("results/hrnet+det")

LR = 1e-4  # transfer learning이니깐 좀 작게 주는게 좋을 것 같아서 1e-4
BATCH_SIZE = 10
START_EPOCH = 1
PADDING = 30
ADD_JOINT_LOSS = False

INPUT_WIDTH = 576  # 288
INPUT_HEIGHT = 768  # 384

n = datetime.now()
UID = f"{n.year:04d}{n.month:02d}{n.day:02d}-{n.hour:02d}{n.minute:02d}{n.second:02d}"
SEED = 20210309

In [4]:
torch.set_grad_enabled(False)

---

## 데이터셋 준비

### Train / Validation

In [5]:
total_imgs = np.array(sorted(list(Path("data/ori/train_imgs/").glob("*.jpg"))))
test_imgs = np.array(sorted(list(Path("data/ori/test_imgs/").glob("*.jpg"))))

df = pd.read_csv("data/ori/train_df.csv")
total_keypoints = df.to_numpy()[:, 1:].astype(np.float32)
total_keypoints = np.stack([total_keypoints[:, 0::2], total_keypoints[:, 1::2]], axis=2)

# 오류가 있는 데이터는 학습에서 제외
total_imgs_, total_keypoints_ = [], []
for i in range(len(total_imgs)):
    if i not in error_list:
        total_imgs_.append(total_imgs[i])
        total_keypoints_.append(total_keypoints[i])
total_imgs = np.array(total_imgs_)
total_keypoints = np.array(total_keypoints_)

In [18]:
class KeypointDataset(Dataset):
    def __init__(self, files, keypoints, augmentation=True, padding=30):
        super().__init__()
        self.files = files
        self.keypoints = keypoints
        self.padding = padding

        T = []
        # T.append(A.Crop(0, 28, 1920, 1080 - 28))  # 1920x1080 --> 1920x1024
        # T.append(A.Resize(512, 1024))
        if augmentation:
            T.append(A.ImageCompression())
            # T.append(A.ShiftScaleRotate(border_mode=cv2.BORDER_CONSTANT, value=0, rotate_limit=0))
            # T.append(utils.HorizontalFlipEx())
            T.append(A.Cutout())
            T_ = []
            T_.append(A.RandomBrightnessContrast())
            T_.append(A.RandomGamma())
            T_.append(A.RandomBrightness())
            T_.append(A.RandomContrast())
            T.append(A.OneOf(T_))
            T.append(A.GaussNoise())
            T.append(A.Blur())
        T.append(A.Normalize())
        T.append(ToTensorV2())

        self.transform = A.Compose(
            transforms=T,
            bbox_params=A.BboxParams(format="pascal_voc", label_fields=["labels"]),
            keypoint_params=A.KeypointParams(format="xy", remove_invisible=False),
            # TODO 영역을 벗어난 keypoint는 그 영역의 한도 값으로 설정해줄 것?
        )

    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        file = str(self.files[idx])
        image = imageio.imread(file)

        keypoint = self.keypoints[idx]
        box = utils.keypoint2box(keypoint, self.padding)
        box = np.expand_dims(box, 0)
        labels = np.array([0], dtype=np.int64)
        a = self.transform(image=image, labels=labels, bboxes=box, keypoints=keypoint)

        image = a["image"]
        bbox = list(map(int, a["bboxes"][0]))
        keypoint = torch.tensor(a["keypoints"], dtype=torch.float32)
        image, keypoint, heatmap, ratio = self._resize_image(image, bbox, keypoint)

        return file, image, keypoint, heatmap, ratio

    def _resize_image(self, image, bbox, keypoint):
        # efficientdet에서 찾은 범위만큼 이미지를 자름
        image = image[:, bbox[1] : bbox[3], bbox[0] : bbox[2]]

        # HRNet의 입력 이미지 크기로 resize
        ratio = torch.tensor((INPUT_WIDTH / image.shape[2], INPUT_HEIGHT / image.shape[1]), dtype=torch.float32)
        image = F.interpolate(image.unsqueeze(0), (INPUT_HEIGHT, INPUT_WIDTH))[0]

        # bbox만큼 빼줌
        keypoint[:, 0] -= bbox[0]
        keypoint[:, 1] -= bbox[1]

        # 이미지를 resize해준 비율만큼 곱해줌
        keypoint[:, 0] *= ratio[0]
        keypoint[:, 1] *= ratio[1]
        # TODO: 잘못된 keypoint가 있으면 고쳐줌

        # HRNet은 1/4로 resize된 출력이 나오므로 4로 나눠줌
        keypoint /= 4

        # keypoint를 heatmap으로 변환
        # TODO: 완전히 정답이 아니면 틀린 것과 같은 점수. 좀 부드럽게 만들 수는 없을지?
        # heatmap regression loss중에 soft~~~ 한 이름이 있던거같은데
        heatmap = utils.keypoints2heatmaps(keypoint, INPUT_HEIGHT // 4, INPUT_WIDTH // 4)

        return image, keypoint, heatmap, ratio

### Test

In [7]:
test_imgs = np.array(sorted(list(Path("data/test_imgs_effdet/").glob("*.jpg"))))

with open("data/test_imgs_effdet/data.json", "r") as f:
    data = json.load(f)
    offsets = data["offset"]
    ratios = data["ratio"]

In [8]:
class TestKeypointDataset(Dataset):
    def __init__(self, files, offsets, ratios, augmentation=False):
        super().__init__()
        self.files = files
        self.offsets = offsets
        self.ratios = ratios

        T = []
        if augmentation:
            T.append(A.ImageCompression())
            T.append(A.Cutout())
            T_ = []
            T_.append(A.RandomBrightnessContrast())
            T_.append(A.RandomGamma())
            T_.append(A.RandomBrightness())
            T_.append(A.RandomContrast())
            T.append(A.OneOf(T_))
            T.append(A.GaussNoise())
            T.append(A.Blur())
        T.append(A.Normalize())
        T.append(ToTensorV2())

        self.transform = A.Compose(transforms=T)

    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        file = str(self.files[idx])
        offset = torch.tensor(self.offsets[idx], dtype=torch.float32)
        ratio = torch.tensor(self.ratios[idx], dtype=torch.float32)

        image = imageio.imread(file)
        a = self.transform(image=image)
        image = a["image"]

        return file, image, offset, ratio

---

## 학습 준비

In [9]:
class JointMSELoss(nn.Module):
    def __init__(self, use_target_weight=False):
        super(JointMSELoss, self).__init__()
        self.criterion = nn.MSELoss(reduction="mean")
        self.use_target_weight = use_target_weight

    def forward(self, output, target, target_weight=None):
        batch_size = output.size(0)
        num_joints = output.size(1)
        heatmaps_pred = output.reshape((batch_size, num_joints, -1)).split(1, 1)
        heatmaps_gt = target.reshape((batch_size, num_joints, -1)).split(1, 1)
        loss = 0

        for idx in range(num_joints):
            heatmap_pred = heatmaps_pred[idx].squeeze()
            heatmap_gt = heatmaps_gt[idx].squeeze()
            if self.use_target_weight:
                loss += 0.5 * self.criterion(heatmap_pred.mul(target_weight[:, idx]), heatmap_gt.mul(target_weight[:, idx]))
            else:
                loss += 0.5 * self.criterion(heatmap_pred, heatmap_gt)

        return loss / num_joints

In [10]:
class KeypointLoss(nn.Module):
    def __init__(self, joint=False, use_target_weight=False):
        super().__init__()
        self.criterion = nn.MSELoss(reduction="mean")
        self.joint = joint
        self.use_target_weight = use_target_weight

    def forward(self, x, y):
        x = x.flatten(2).flatten(0, 1)
        y = y.flatten(2).flatten(0, 1).argmax(1)
        loss1 = F.cross_entropy(x, y)

        if self.joint:
            loss2 = self.joint_mse_loss(x, y)
            return loss1 + loss2
        else:
            return loss1

    def joint_mse_loss(self, output, target, target_weight=None):
        batch_size = output.size(0)
        num_joints = output.size(1)
        heatmaps_pred = output.reshape((batch_size, num_joints, -1)).split(1, 1)
        heatmaps_gt = target.reshape((batch_size, num_joints, -1)).split(1, 1)
        loss = 0

        for idx in range(num_joints):
            heatmap_pred = heatmaps_pred[idx].squeeze()
            heatmap_gt = heatmaps_gt[idx].squeeze()
            if self.use_target_weight:
                loss += 0.5 * self.criterion(heatmap_pred.mul(target_weight[:, idx]), heatmap_gt.mul(target_weight[:, idx]))
            else:
                loss += 0.5 * self.criterion(heatmap_pred, heatmap_gt)

        return loss / num_joints

In [11]:
class KeypointRMSE(nn.Module):
    @torch.no_grad()
    def forward(self, pred_heatmaps: torch.Tensor, real_heatmaps: torch.Tensor, ratios: torch.Tensor):
        W = pred_heatmaps.size(3)
        pred_positions = pred_heatmaps.flatten(2).argmax(2)
        real_positions = real_heatmaps.flatten(2).argmax(2)
        pred_positions = torch.stack((pred_positions // W, pred_positions % W), 2).type(torch.float32)
        real_positions = torch.stack((real_positions // W, real_positions % W), 2).type(torch.float32)
        # print(pred_positions.shape, real_positions.shape, ratios.shape)
        pred_positions *= 4 / ratios.unsqueeze(1)  # position: (B, 24, 2), ratio: (B, 2)
        real_positions *= 4 / ratios.unsqueeze(1)
        loss = (pred_positions - real_positions).square().mean().sqrt()

        """
        W = x.size(3)
        xp = x.flatten(2).argmax(2)
        xx = (xp % W) / ratios[:, 0:1] * 4
        xy = (xp // W) / ratios[:, 1:2] * 4
        yp = y.flatten(2).argmax(2)
        yx = (yp % W) / ratios[:, 0:1] * 4
        yy = (yp // W) / ratios[:, 1:2] * 4

        diff = ((xx - yx) ** 2 + (xy - yy) ** 2) / 2
        loss = diff.mean().sqrt()
        """
        return loss

In [12]:
@dataclass
class TrainOutputBean:
    loss = utils.AverageMeter()
    rmse = utils.AverageMeter()

    def freeze(self):
        self.loss = self.loss()
        self.rmse = self.rmse()

        return self

In [13]:
class TrainInputBean:
    def __init__(self, evaluation_mode=False):
        self.evaluation_mode = evaluation_mode

        # HRNet 생성
        if POSE_MODEL == "HRNet-W32":
            width = 32
        elif POSE_MODEL == "HRNet-W48":
            width = 48
        else:
            raise NotImplementedError()

        self.pose_model = networks.PoseHighResolutionNet(width)
        self.pose_model.load_state_dict(torch.load(f"networks/models/pose_hrnet_w{width}_384x288.pth"))

        final_layer = nn.Conv2d(width, 24, 1)
        with torch.no_grad():
            final_layer.weight[:17] = self.pose_model.final_layer.weight
            final_layer.bias[:17] = self.pose_model.final_layer.bias
            self.pose_model.final_layer = final_layer
        self.pose_model.cuda()

        # Criterion / Optimizer
        # self.criterion = JointMSELoss().cuda()
        self.criterion = KeypointLoss().cuda()
        self.criterion_rmse = KeypointRMSE().cuda()

        self.epoch = START_EPOCH

        if not self.evaluation_mode:
            if SAM:
                self.optimizer = utils.SAM(self.pose_model.parameters(), optim.AdamW, lr=LR)
            else:
                self.optimizer = optim.AdamW(self.pose_model.parameters(), lr=LR)
            self.scheduler = ReduceLROnPlateau(self.optimizer, factor=0.5, patience=4, verbose=True)

            # 기타
            self.best_loss = math.inf
            self.earlystop_cnt = 0

    def save(self, path):
        torch.save(
            {
                "model": self.pose_model.state_dict(),
                "optimizer": self.optimizer.state_dict(),
                "epoch": self.epoch,
                "best_loss": self.best_loss,
                "earlystop_cnt": self.earlystop_cnt,
            },
            path,
        )

    def load(self, path):
        print("Load pretrained", path)
        ckpt = torch.load(path)
        self.pose_model.load_state_dict(ckpt["model"])

        if not self.evaluation_mode:
            self.optimizer.load_state_dict(ckpt["optimizer"])
            self.epoch = ckpt["epoch"]
            self.best_loss = ckpt["best_loss"]
            self.earlystop_cnt = ckpt["earlystop_cnt"]

In [14]:
@dataclass
class TrainOutputBean:
    loss = utils.AverageMeter()
    rmse = utils.AverageMeter()

    def freeze(self):
        self.loss = self.loss()
        self.rmse = self.rmse()

        return self

---

## 모델, 데이터셋 생성

In [19]:
kf = KFold(n_splits=5, shuffle=True, random_state=SEED)
indices = list(kf.split(total_imgs))

train_idx, valid_idx = indices[0]
ds_train = KeypointDataset(
    total_imgs[train_idx],
    total_keypoints[train_idx],
    augmentation=True,
    padding=PADDING,
)
ds_valid = KeypointDataset(
    total_imgs[valid_idx],
    total_keypoints[valid_idx],
    augmentation=False,
    padding=PADDING,
)
dl_train = DataLoader(ds_train, batch_size=BATCH_SIZE, num_workers=4, shuffle=True)
dl_valid = DataLoader(ds_valid, batch_size=BATCH_SIZE, num_workers=4, shuffle=False)

B = TrainInputBean(evaluation_mode=True)
B.load(CHECKPOINT_PATH)

Load pretrained results/hrnet+det/ckpt-20210321-124655_2.pth


In [20]:
ds_test = TestKeypointDataset(test_imgs, offsets, ratios, augmentation=False)
dl_test = DataLoader(ds_test, batch_size=BATCH_SIZE, num_workers=4, shuffle=False)

---

## Validation 1epoch 돌려보기

In [17]:
valid_output_dir = Path("results/hrnet_det_example/valid_1")
shutil.rmtree(valid_output_dir)
valid_output_dir.mkdir(parents=True, exist_ok=True)

torch.cuda.empty_cache()
B.pose_model.eval()
O = TrainOutputBean()
with tqdm(total=len(dl_valid.dataset), ncols=100, leave=True, file=sys.stdout, desc=f"Valid[{B.epoch:03d}]") as t:
    for files, imgs, keypoints, target_heatmaps, ratios in dl_valid:
        imgs_, target_heatmaps_ = imgs.cuda(), target_heatmaps.cuda()
        pred_heatmaps_ = B.pose_model(imgs_)
        loss = B.criterion(pred_heatmaps_, target_heatmaps_)
        rmse = B.criterion_rmse(pred_heatmaps_, target_heatmaps_, ratios.cuda())
        
        O.loss.update(loss.item())
        O.rmse.update(rmse.item())

        # 이미지로 저장
        for file, img, pred_heatmap, target_heatmap, ratio in zip(files, imgs, pred_heatmaps_.cpu(), target_heatmaps, ratios):
            file = Path(file)
            t.set_postfix_str(file.name)
            
            pred_keypoint = utils.heatmaps2keypoints(pred_heatmap)
            target_keypoint = utils.heatmaps2keypoints(target_heatmap)
            pred_keypoint = pred_keypoint * 4 / ratio.view(1,2)
            target_keypoint = target_keypoint * 4 / ratio.view(1,2)
            pred_keypoint = pred_keypoint.type(torch.int64).numpy()
            target_keypoint = target_keypoint.type(torch.int64).numpy()
            
            img_np = utils.denormalize(img).mul(255).type(torch.uint8).permute(1,2,0).numpy()
            ori_width = img.size(2) / ratio[0]
            ori_height = img.size(1) / ratio[1]
            img_np = cv2.resize(img_np, (ori_width, ori_height))
            
            pred_keypoint_img = utils.draw_keypoints(img_np, pred_keypoint)
            target_keypoint_img = utils.draw_keypoints(img_np, target_keypoint)
            
            rmse = np.sqrt(np.mean(np.square(pred_keypoint - target_keypoint)))
            
            imageio.imwrite(valid_output_dir/f"{file.name[:-4]}_pred_{rmse:.4f}.jpg", pred_keypoint_img)
            imageio.imwrite(valid_output_dir/f"{file.name[:-4]}_target.jpg", target_keypoint_img)

            t.update()
O.freeze()
pass

In [33]:
utils.heatmaps2keypoints(pred_heatmap)

tensor([[121,  15],
        [125,  11],
        [114,  11],
        [ 94,   9],
        [ 93,   9],
        [119,  33],
        [ 61,  32],
        [122,  55],
        [ 17,  52],
        [125,  74],
        [ 46,  77],
        [110,  95],
        [ 73,  95],
        [106, 137],
        [ 67, 137],
        [102, 173],
        [ 70, 177],
        [ 95,  23],
        [126,  78],
        [ 56,  81],
        [ 94,  52],
        [ 93,  77],
        [113, 178],
        [ 85, 183]])

In [37]:
train_idx[:20], valid_idx

(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 17,
        18, 20, 21]),
 array([  16,   19,   22,   25,   26,   29,   41,   44,   46,   52,   53,
          58,   61,   71,   99,  100,  102,  104,  105,  107,  108,  109,
         121,  126,  129,  136,  138,  144,  152,  158,  164,  165,  168,
         173,  179,  186,  191,  196,  206,  216,  223,  226,  236,  242,
         244,  246,  251,  255,  257,  267,  273,  274,  300,  303,  304,
         313,  322,  344,  362,  363,  369,  370,  372,  377,  392,  395,
         396,  398,  404,  410,  411,  412,  418,  422,  424,  427,  431,
         442,  445,  448,  450,  452,  456,  457,  462,  467,  471,  473,
         480,  484,  489,  493,  495,  496,  508,  509,  531,  543,  544,
         548,  550,  554,  560,  569,  571,  573,  577,  580,  581,  584,
         586,  590,  592,  602,  608,  611,  612,  615,  616,  617,  619,
         620,  627,  628,  633,  639,  642,  645,  650,  658,  661,  663,
         675, 

---

## submission 만들기

In [12]:
for files, imgs, offsets, ratios in dl_test:
    break

<IPython.core.display.Javascript object>

In [32]:
ratios.shape

torch.Size([40, 2])

In [43]:
file_names = []
result = []
with tqdm(total=len(dl_test.dataset), ncols=100, file=sys.stdout) as t:
    for files, imgs, offsets, ratios in dl_test:
        pred_heatmaps = B.pose_model(imgs.cuda()).cpu()
        for file, pred_heatmap, offset, ratio in zip(files, pred_heatmaps, offsets, ratios):
            file = Path(file)
            file_names.append(file.name)
            t.set_postfix_str(file.name)

            keypoint = utils.heatmaps2keypoints(pred_heatmap)
            keypoint = keypoint * 4 / ratio.view(1, 2) + offset.view(1, 2)

            # effdet의 ROI
            keypoint[:, 1] += 28
            keypoint[:, 0] *= 1920 / 1024
            keypoint[:, 1] *= 1024 / 512
            result.append(keypoint)
            t.update()

100%|████████████████████████████| 1600/1600 [00:13<00:00, 122.18it/s, 786-3-5-41-Z94_E-0000031.jpg]


In [45]:
file_names = np.array(file_names)

In [47]:
result = torch.stack(result)

In [67]:
submit_data = np.concatenate([np.expand_dims(file_names, 1), result.flatten(1).numpy()], 1)

In [70]:
sample_submission = pd.read_csv("data/ori/sample_submission.csv")

In [71]:
submit_df = pd.DataFrame(submit_data, columns=sample_submission.columns)

In [72]:
submit_df

,image,nose_x,nose_y,left_eye_x,left_eye_y,right_eye_x,right_eye_y,left_ear_x,left_ear_y,right_ear_x,...,right_palm_x,right_palm_y,spine2(back)_x,spine2(back)_y,spine1(waist)_x,spine1(waist)_y,left_instep_x,left_instep_y,right_instep_x,right_instep_y
0,649-2-4-32-Z148_A-0000001.jpg,1097.2135,545.9375,1097.2135,565.8541,1119.6354,560.875,1067.3176,620.625,1157.0052,...,977.6302,316.89584,977.6302,555.8959,925.3125,521.0416,678.6719,725.1875,820.6771,680.375
1,649-2-4-32-Z148_A-0000003.jpg,1107.8126,544.875,1100.2865,564.7084,1122.8645,559.75,1070.1824,619.25,1160.4948,...,1017.50006,401.0833,979.8698,554.7916,912.13544,530.0,678.8281,723.375,821.8229,678.75
2,649-2-4-32-Z148_A-0000005.jpg,1096.0156,546.1666,1103.4375,561.2916,1118.2812,561.2916,1066.3282,621.7916,1155.3907,...,1014.375,314.25,955.0,566.3334,888.2031,551.2084,680.3906,722.625,821.40625,682.2916
3,649-2-4-32-Z148_A-0000007.jpg,1111.7188,562.2292,1084.375,571.5417,1148.177,549.8125,1047.9167,611.8959,1148.177,...,1257.5521,608.7917,1084.375,667.7709,902.0833,531.1875,674.21875,723.6459,820.05206,680.1875
4,649-2-4-32-Z148_A-0000009.jpg,1097.552,543.9375,1097.552,563.8541,1119.8959,314.89584,1067.7604,623.6041,1157.1355,...,1105.0,334.8125,956.0417,558.875,896.4583,548.9166,680.46875,723.1875,821.9792,678.375
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1595,786-3-5-41-Z94_E-0000023.jpg,691.0416,601.4583,684.6875,593.25,684.6875,589.1458,633.8541,580.9375,703.74994,...,868.9584,736.8959,640.2084,687.6459,697.3959,720.4792,938.8542,913.375,983.3334,872.3334
1596,786-3-5-41-Z94_E-0000025.jpg,931.6667,622.75,911.4844,611.5,911.4844,611.5,915.5208,600.25,935.7031,...,1056.7969,750.25,963.95825,776.5,846.901,720.25,947.8125,900.25,947.8125,900.25
1597,786-3-5-41-Z94_E-0000027.jpg,687.3958,588.1667,748.6979,572.0833,709.6875,596.2083,698.5416,584.1458,726.40625,...,849.0104,724.875,787.7083,636.4167,720.8334,636.4167,938.177,909.8334,960.46875,889.7292
1598,786-3-5-41-Z94_E-0000029.jpg,847.7344,623.625,875.9896,563.9375,884.0625,766.875,847.7344,587.8125,855.80725,...,968.82806,731.0625,815.4427,643.5208,811.40625,830.5417,997.0833,906.1459,960.7552,902.1667


In [73]:
submit_df.describe()

,image,nose_x,nose_y,left_eye_x,left_eye_y,right_eye_x,right_eye_y,left_ear_x,left_ear_y,right_ear_x,...,right_palm_x,right_palm_y,spine2(back)_x,spine2(back)_y,spine1(waist)_x,spine1(waist)_y,left_instep_x,left_instep_y,right_instep_x,right_instep_y
count,1600,1600,1600,1600,1600,1600,1600,1600,1600,1600,...,1600,1600,1600,1600,1600,1600,1600,1600,1600,1600
unique,1600,1322,1350,1367,1373,1350,1356,1342,1333,1375,...,1382,1414,1337,1360,1308,1340,1340,1337,1316,1287
top,754-3-5-38-Z94_A-0000029.jpg,941.25,554.0,1171.875,502.0,1163.4375,494.9375,1049.349,602.0,1171.875,...,849.84375,304.25,942.1875,565.5625,942.1875,605.5,741.5625,918.6875,640.3125,826.0
freq,1,11,9,7,9,7,6,6,7,9,...,6,8,9,8,9,7,8,8,8,8


In [74]:
submit_df.to_csv("results/hrnet+det/submit-20210320-114245_1.csv", index=False)